In [ ]:
%pip install -i https://test.pypi.org/simple/ cryptogame==0.0.9

# Integrity of Ciphertext (INT-CTXT)

This section consists of SE IND-CPA and SE INT-CTXT security games

In [ ]:
from cryptogame.tools.utils import random_bits
from cryptogame.tools.AES import *
from cryptogame.game.indcpa import *
from cryptogame.game.int_ctxt import *
from simulate import *

Define $SE = (\mathcal{K},\mathcal{E},\mathcal{D})$ where $\mathcal{K}$ returns a random 128-bit key K and:<br>

**Algorithm** $\mathcal{E}_K(M)$<br>
&nbsp;&nbsp;&nbsp;&nbsp;If $|M|\neq 512$ then return $\perp$<br>
&nbsp;&nbsp;&nbsp;&nbsp;$M[1]||M[2]||M[3]||M[4]\leftarrow M$<br>
&nbsp;&nbsp;&nbsp;&nbsp;$C_e[0]\leftarrow\!\tiny{\$}$ $\{0,1\}^{128};C_m[0]\leftarrow0^{128}$<br>
&nbsp;&nbsp;&nbsp;&nbsp;For $i=1,2,3,4$ do:<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$C_e[i]\leftarrow AES_K(C_e[i-1]\oplus M[i])$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$C_m[i]\leftarrow AES_K(C_m[i-1]\oplus M[i])$<br>
&nbsp;&nbsp;&nbsp;&nbsp;$C_e\leftarrow C_e[0]||C_e[1]||C_e[2]||C_e[3]||C_e[4]$<br>
&nbsp;&nbsp;&nbsp;&nbsp;$T\leftarrow C_m[4];$ Return $(C_e,T)$<br>

**Algorithm** $\mathcal{D}_K((C_e,T))$:<br>
&nbsp;&nbsp;&nbsp;&nbsp;If $|C_e|\neq 640$ then return $\perp$<br>
&nbsp;&nbsp;&nbsp;&nbsp;$C_m[0] \leftarrow 0^{128}$<br>
&nbsp;&nbsp;&nbsp;&nbsp;For $i=1,2,3,4$ do:<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$M[i]\leftarrow AES_K^{-1}(C_e[i])\oplus C_e[i-1]$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$C_m[i]\leftarrow AES_K(C_m[i-1]\oplus M[i])$<br>
&nbsp;&nbsp;&nbsp;&nbsp;If $C_m[4]=T$ then return $M[1]||M[2]||M[3]||M[4]$<br>
&nbsp;&nbsp;&nbsp;&nbsp;Else return $\perp$<br>

Initialize encoder $\mathcal{E}_K$ and decoder $\mathcal{D}_K$ below:

In [ ]:
key_len = 128
block_len = 128
aes = AES(key_len, block_len)

def enc(key, m):
	if len(m) != 512:
		return None
	
	ce_0 = random_bits(128)
	cm_0 = '0'*128
	c = ce_0
	for i in range(4):
		encrypted_block =  aes.evaluate(key, xor(ce_0,m[i*block_len: (i+1)*block_len]))
		ce_0 = encrypted_block  ## updating ce_0 for ce[i-1]
		c += encrypted_block

		cm_i = aes.evaluate(key, xor(cm_0, m[i*block_len: (i+1)*block_len]))
		cm_0 = cm_i
	return (c, cm_i)

def dec(key, c):
	cipher, t = c
	if len(cipher) != 640:
		return None
	cm_0 = '0'*128
	ce_0 = cipher[:block_len]
	ce = cipher[block_len:]
	m_i = str()
	for i in range(4):
		decrypted_block = xor(aes.inverse(key, ce[i*block_len: (i+1)*block_len]), ce_0) 
		m_i += decrypted_block
		ce_0 = ce[i*block_len: (i+1)*block_len]

		cm_i = aes.evaluate(key, xor(cm_0, decrypted_block))
		cm_0 = cm_i
	if cm_i == t:
		return m_i
	else:
		return None
	


### TODO: SE IND-CPA:

Is SE IND-CPA secure?<br>
If not, provide an adversary in $adv\_indcpa$ below

In [ ]:
### Is SE IND-CPA secure?

game_indcpa = INDCPA(enc, dec, 128, 128, 10)
game_indcpa._initialize()

def adv_indcpa(game):
	#========= Your Code goes here =======================
	x = random_bits(512)
	y = random_bits(512)

	q1 = game.LR(x, x)
	c1, t1 = q1

	q2 = game.LR(x, y)
	c2, t2 = q2
	# =====================================================
	if t1 == t2:
		return 1
	else: 
		return 0
	
game_indcpa.finalize(adv_indcpa(game_indcpa))
s = Simulate(game_indcpa, adv_indcpa)
s.simulate_INDCPA(verbose=True)

### Inline Question #1:

Is SE IND-CPA Secure? If so, provide a proof.

**Response**:

### TODO: SE INT-CTXT:

Is SE INT-CTXT secure?<br>
If not, provide an adversary in $adv\_intctxt$ below

In [ ]:
game_intctxt = INTCTXT(enc, dec, 128, 20)
game_intctxt._initialize()

def adv_intctxt(game: INTCTXT):
	#========= Your Code goes here =======================
	x = '0'*512
	cipher, tag = game.Enc(x)
	c_0 = '0'*128
	c_1 = cipher[1*128:2*128]
	c_2 = cipher[2*128: 3*128]
	c_3 = cipher[3*128 : 4*128]
	c_4 = cipher[4*128: 5*128]
	c_prime = c_0 + c_1 + c_2 + c_3 + c_4
	t_prime = c_4
	# =====================================================
	return (c_prime, t_prime)

game_intctxt.finalize(adv_intctxt(game_intctxt))
s = Simulate(game_intctxt, adv_intctxt)
s.simulate_intctxt(verbose= True)


### Inline Question #2:

Is SE INT-CTXT Secure? If so, provide a proof.

**Response**: